In [7]:
import sqlite3 as sq

In [8]:
conn = sq.connect('my_db.db')
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS получатель(
    код_получателя INT PRIMARY KEY,
    фамилия TEXT,
    имя TEXT,
    отчество TEXT,
    дата_рождения TEXT,
    индекс INT,
    город TEXT,
    улица TEXT,
    дом INT,
    квартира INT,
    телефон TEXT);
''')
conn.commit()

In [9]:
cur.execute('''CREATE TABLE IF NOT EXISTS курьер(
    код_курьера INT PRIMARY KEY,
    фамилия TEXT,
    имя TEXT,
    отчество TEXT,
    номер_паспорта INT,
    дата_рождения TEXT,
    дата_приема_на_работу TEXT,
    начало_рабочего_дня TEXT,
    конец_рабочего_дня TEXT,
    город TEXT,
    улица TEXT,
    дом INT,
    квартира INT,
    телефон TEXT);
''')
conn.commit()

In [10]:
courier = ('01', 'Иванов', 'Иван', 'Иванович', '123123','10-10-2000','29-11-2024','8:00','16:00','Калиниград','Пушкина','10','10','8-800-555-35-35')
recipient = ('01', 'Степанов', 'Степан', 'Степанович', '05-05-2005', '23017','Калининград','Степана Разина','9','9','8-918-123-12-12')

cur.execute("INSERT INTO получатель VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", recipient)
conn.commit()

cur.execute("INSERT INTO курьер VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", courier)
conn.commit()

In [14]:
cur.execute("SELECT * from курьер;")
a = cur.fetchone()
print(a)

(1, 'Иванов', 'Иван', 'Иванович', 123123, '10-10-2000', '29-11-2024', '8:00', '16:00', 'Калиниград', 'Пушкина', 10, 10, '8-800-555-35-35')


In [15]:
cur.execute("SELECT * from получатель;")
a = cur.fetchone()
print(a)

(1, 'Степанов', 'Степан', 'Степанович', '05-05-2005', 23017, 'Калининград', 'Степана Разина', 9, 9, '8-918-123-12-12')


In [16]:
cur.execute('''
    UPDATE получатель
    SET город = ?, улица = ?, дом = ?, квартира = ?
    WHERE код_получателя = ?;
''', ('Москва', 'Ленина', '15', '12', '1'))
conn.commit()

In [17]:
cur.execute("SELECT * from получатель;")
a = cur.fetchone()
print(a)

(1, 'Степанов', 'Степан', 'Степанович', '05-05-2005', 23017, 'Москва', 'Ленина', 15, 12, '8-918-123-12-12')


In [18]:
conn.close()

2 задание


In [20]:
%pip install peewee

     ---------------------------------------- 0.0/948.2 kB ? eta -:--:--
     --------------------- ---------------- 524.3/948.2 kB 5.6 MB/s eta 0:00:01
     -------------------------------------- 948.2/948.2 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=139012 sha256=1eb67b574d914058131866745c2f2118c5d93f7228af9fc87ec783b94cdb8639
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\8f\65\34\456800445efeafb05164fe95285c70e81ba1d96bae30f43917
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from peewee import*

db = SqliteDatabase('my_db.db')

class транспорт(Model):
    номер_машины = IntegerField(primary_key = True)
    марка = TextField()
    дата_регистрации = TextField()
    цвет = TextField()

    class Meta:
        database = db
        table_name = 'транспорт'


class отправитель(Model):
    код_отправителя = IntegerField(primary_key = True)
    фамилия = TextField()
    имя = TextField()
    отчество = TextField()
    дата_рождения = TextField()
    индекс = IntegerField()
    город = TextField()
    улица = TextField()
    дом = IntegerField()
    квартира = IntegerField()
    телефон = TextField()
    
    class Meta:
        database = db
        table_name = 'отправитель'

In [25]:
class получатель(Model):  ##доопределим тк peewee ругается когда в связях пытаюсь ссылаться 
                          ##на класс таблицы, которую модуль peewee не видел
    код_получателя = IntegerField(primary_key=True)
    фамилия = TextField()
    имя = TextField()
    отчество = TextField()
    дата_рождения = TextField()
    индекс = IntegerField()
    город = TextField()
    улица = TextField()
    дом = IntegerField()
    квартира = IntegerField()
    телефон = TextField()

    class Meta:
        database = db
        table_name = 'получатель'  

class курьер(Model):
    код_курьера = IntegerField(primary_key=True)  
    фамилия = TextField()
    имя = TextField()
    отчество = TextField()
    номер_паспорта = IntegerField()
    дата_рождения = TextField()
    дата_приема_на_работу = TextField()
    начало_рабочего_дня = TextField()
    конец_рабочего_дня = TextField()
    город = TextField()
    улица = TextField()
    дом = IntegerField()
    квартира = IntegerField()
    телефон = TextField()

    class Meta:
        database = db  
        table_name = 'курьер' 

In [27]:
class заказ(Model):
    код_заказа = IntegerField(primary_key=True)
    отправитель = ForeignKeyField(отправитель, backref='заказы', on_delete='CASCADE')
    получатель = ForeignKeyField(получатель, backref='заказы', on_delete='CASCADE')
    дата_заказа = TextField()
    дата_доставки = TextField()
    цена_доставки = FloatField()
    курьер = ForeignKeyField(курьер, backref='заказы', on_delete='CASCADE')
    транспорт = ForeignKeyField(транспорт, backref='заказы', on_delete='CASCADE')

    class Meta:
        database = db
        table_name = 'заказ'


db.connect()
db.create_tables([курьер, отправитель, получатель, транспорт, заказ], safe=True) ##safe предотвращает пересоздание

In [28]:
car = транспорт.create(номер_машины = '999', марка = 'жигули', дата_регистрации = '11-03-2020', цвет = 'черный')
car.save()
sender = отправитель.create(код_отправителя = '122', фамилия= 'Борецкий', имя = 'Станислав', отчество = 'Игоревич', дата_рождения = '21-05-2001', индекс = '444', 
                          город = 'Калининград',  улица = 'Попова', дом = '1 ', квартира = '11', телефон = '8-900-111-11-11')
sender.save()

1

In [31]:
curier = курьер.select().first()
print(curier.фамилия)
recipient = получатель.select().first()
print(recipient.фамилия)

Иванов
Степанов


In [32]:
order = заказ.create(код_заказа = '555', отправитель = sender,получатель = recipient, дата_заказа = '29-11-2024', дата_доставки = '30-11-2024',
                    цена_доставки = '100.12', курьер = curier, транспорт = car)
order.save()


1

In [33]:
got_order = заказ.select().first()
print(f"Код заказа: {got_order.код_заказа}")
print(f"Отправитель: {got_order.отправитель.фамилия} {got_order.отправитель.имя} {got_order.отправитель.отчество}")
print(f"Получатель: {got_order.получатель.фамилия} {got_order.получатель.имя} {got_order.получатель.отчество}")
print(f"Дата заказа: {got_order.дата_заказа}")
print(f"Дата доставки: {got_order.дата_доставки}")
print(f"Цена доставки: {got_order.цена_доставки}")
print(f"Курьер: {got_order.курьер.фамилия} {got_order.курьер.имя} {got_order.курьер.отчество}")
print(f"Транспорт: {got_order.транспорт.марка}, Цвет: {got_order.транспорт.цвет}")

Код заказа: 555
Отправитель: Борецкий Станислав Игоревич
Получатель: Степанов Степан Степанович
Дата заказа: 29-11-2024
Дата доставки: 30-11-2024
Цена доставки: 100.12
Курьер: Иванов Иван Иванович
Транспорт: жигули, Цвет: черный
